This file estimates the ATE for different models. To apply to other types of data frames, simply change the saved file from the simulation. You need also to make sure the graph is the right DAG for the model

In [ ]:
import pandas as pd
from irand.estimation import *
import pickle
import time

## BMI-like no Hidden confounders

In [ ]:
with open('../output_data/BMIlike_noH.pickle', 'rb') as handle:
    dict_df = pickle.load(handle)
outcome = "Y"
treatment = "T"
common_causes = ["X0", "X1"]
LCD_like = False
graph = "digraph{{X0,X1,T}->Y;X0->T; X1->T}"

list_sigmaSq = list(dict_df.keys())
list_seeds = list(dict_df[0.25].keys())
list_sample_size = [100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500]

## Estimating the ATE for each method separately

In [ ]:
df_0 = dict_df[4][list_seeds[10]]['df_0']
df_1 = dict_df[4][list_seeds[10]]['df_1']
df_0['ID'] = df_1.index
df_1['ID'] = df_1.index

# DiD

df_did = create_did(df_0, df_1, no_control=False)
causal_estimate_did = causal_estimate_f(graph, df_did, treatment, outcome, common_causes)

# Pooled Regression

df_pooled = create_pooled(df_0, df_1)
causal_estimate_pooled = causal_estimate_f(graph, df_pooled, treatment, outcome, common_causes)

# iRand
irand_list_ate = i_rand_parallel(df_0, df_1, graph,  
                                 500, 
                                 list_seeds[400], 
                                 treatment, 
                                 outcome, 
                                 common_causes)

In [ ]:
df_0 = dict_df[4][list_seeds[10]]['df_0']
df_1 = dict_df[4][list_seeds[10]]['df_1']
df_0['ID'] = df_1.index
df_1['ID'] = df_1.index

# DiD

df_did = create_did(df_0, df_1, no_control=False)
causal_estimate_did = causal_estimate_f(graph, df_did, treatment, outcome, common_causes)

# Pooled Regression

df_pooled = create_pooled(df_0, df_1)
causal_estimate_pooled = causal_estimate_f(graph, df_pooled, treatment, outcome, common_causes)

# iRand
irand_list_ate = i_rand_parallel(df_0, df_1, graph,  
                                 500, 
                                 list_seeds[400], 
                                 treatment, 
                                 outcome, 
                                 common_causes)

In [ ]:
print('DiD estimate = ' + str(round(causal_estimate_did.value, 3)))
print('DiD estimate = ' + str(round(causal_estimate_pooled.value, 3)))
print('DiD estimate = ' + str(round(irand_list_ate, 3)))

## Checking values for different simulation paths

In [ ]:
results = Parallel(n_jobs=-1)(delayed(calculate_ATEs)(
    dict_df, 0.25, 100, simul_index, graph, treatment, outcome, common_causes, LCD_like) 
                              for simul_index in list_seeds)

# Creating a data frame of the results
pd.DataFrame(results, columns=['sigma_Sq', 'Sample Size', 
                               'Simul Index', 'Pooled ATE', 'DiD ATE', 'iRand ATE'])